# Homework 4: Optimal Control
Due: May 14, 2025

## Preliminaries
Given the dynamics of a system $\dot{x}(t) = f(t, x(t), u(t))$ and the initial state $x(0)=x_0$. We want to calculate the optimal control signal $u(t)$ for the system to optimally track a desired trajectory $x_d(t)$ within the time horizon $[0,T]$.

$$
\begin{align}
u(t)^* &= \argmin_{u(t)} J(u(t)) \\
&= \argmin_{u(t)} \int_0^T \left[\underbrace{\left(x(t)-x_d(t)\right)^T Q_x\left(x(t)-x_d(t)\right) + u(t)^T R_u(t)}_{l(x(t),u(t))}\right] dt + \underbrace{\left(x(T) - x_d(T)\right)^T P_1\left(x(T)-x_d(T)\right)}_{m(x(T))} \\
\text{s.t. } \,\, &x(t) = x_0 + \int_0^t f\left(x(\tau), u(\tau)\right) d\tau
\end{align}
$$
We can solve this problem using iLQR. In the k-th interation, given the current estimation of the optimal control $u(t)^{[k]}$ and the corresponding system trajectory $x(t)^{[k]}$, we need to calculate the optimal descent direction, denoted as $v(t)^{[k]}$, through another optimal control problem below:

$$
v(t)^{[k]} = \argmin_{v(t)}\underbrace{\int_0^T \underbrace{D_1 l(x(t)^{[k]}, u(t)^{[k]})}_{a_x(t)}\cdot z(t) + \underbrace{D_2 l(x(t)^{[k]}, u(t)^{[k]})}_{b_u(t)}\cdot v(t) dt + \underbrace{D m(x(T)^{[k]})}_{p_1}\cdot z(T)}_{DJ(u(t))\cdot v(t)} + \int_0^T z(t)^T Q_z z(t) + v(t)^T R_v v(t) dt
$$

where $z(t)$ and $v(t)$ are governed by the following linear dynamics:

$$
z(t) = \underbrace{z_0}_{z_0=0} + \int_0^t \underbrace{D_1 f(x(\tau)^{[k]}, u(\tau)^{[k]})}_{A(\tau)}\cdot z(\tau) + \underbrace{D_2 f(x(\tau)^{[k]}, u(\tau)^{[k]})}_{B(\tau)} \cdot v(\tau) d\tau
$$

Once the optimal descent direction is calculated, we can use it to update the control for the next iteration using Armijo line search.

# Problem 1
The solution of $v(t)^{[k]}$ can be calculated through the following ODEs:

$$
\begin{align}
0 &= p(t)^T B(t) + b_v(t)^T \\
\dot{p}(t) &= -A(t)^T p(t) - a_z(t) \\
\dot{z}(t) &= A(t)z(t) + B(t)v(t)
\end{align}
$$
with initial and terminal conditions being:

$$
z(0) = 0 \,\,\,\, p(T) = p_1
$$

These ODEs can be reorganized into the following two-point boundary value problem, which does not involve $v(t)$ at all.

$$
\begin{bmatrix}
\dot{z}(t) \\ \dot{p}(t)
\end{bmatrix} = \underbrace{\begin{bmatrix}
M_{11} & M_{12} \\ M_{21} & M_{22}
\end{bmatrix}}_{M} \begin{bmatrix}
z(t) \\ p(t)
\end{bmatrix} + \begin{bmatrix}
m_1 \\ m_2
\end{bmatrix}, \,\,\,\, \begin{bmatrix}
z(0) \\ p(T)
\end{bmatrix} = \begin{bmatrix}
0 \\ p_1
\end{bmatrix}
$$

What should $a_z(t)$ $b_v(t)$ be? Note that they are different from the terms $a_x(t)$ and $b_u(t)$ above. What should the block matrix $M$ look like? What should the vectors $m_1$ and $m_2$ look like? (Hint: the block matrix $M$ and vectors $m_1$ and $m_2$ should not include $v(t)$ at all). Lastly, how to calculate $v(t)$ once you have solved the above two point boundary value problem.

**Turn in:** The expressions for $a_z(t), b_v(t)$, the block matrix $M$, the vectors $m_1$ and $m_2$, and $v(t)$ (assuming $p(t)$ and $z(t)$ are solved).

$$
a_z(t) = 
$$

$$
b_v(t) = 
$$

$$
M = 
$$

$$
m_1 = 
$$

$$
m_2 = 
$$

$$
v(t) = 
$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy